<a href="https://colab.research.google.com/github/gmflau/GenAI/blob/main/colab/Nvidia_NEMO_Embedding_for_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NVIDIA NeMo Retriever provides ‌an embedding service tailored for question-answering applications. The embedding model and service are provided under a commercial use license.

In [7]:
import requests
import numpy as np
from google.colab import userdata

# Make sure to fill this. You will need to obtain the API_KEY from NGC playground
API_KEY=userdata.get('NGC_PLAYGROUND_API_KEY')

invoke_url = "https://api.nvcf.nvidia.com/v2/nvcf/pexec/functions/091a03bb-7364-4087-8090-bd71e9277520"
fetch_url_format = "https://api.nvcf.nvidia.com/v2/nvcf/pexec/status/"
headers = {
    "Authorization": "Bearer {}".format(API_KEY),
    "Accept": "application/json",
}

# To re-use connections
session = requests.Session()

# Note the "model": "passage" field in the payload.
passage_payload = {
  "input": ["Pablo Ruiz Picasso was a Spanish painter, sculptor, printmaker, ceramicist and theater designer who spent most of his adult life in France.",
            "Albert Einstein was a German-born theoretical physicist who is widely held to be one of the greatest and most influential scientists of all time.",
            "J. Robert Oppenheimer was an American theoretical physicist. He was director of the Manhattan Project's Los Alamos Laboratory during World War II and is often called the - father of the atomic bomb"],
    "model" : "passage",
    "encoding_format": "float"
}

passage_response = session.post(invoke_url, headers=headers, json=passage_payload)
while passage_response.status_code == 202:
    request_id = passage_response.headers.get("NVCF-REQID")
    fetch_url = fetch_url_format + request_id
    passage_response = session.get(fetch_url, headers=headers)

passage_response.raise_for_status()
passage_embeddings = np.asarray([item['embedding'] for item in passage_response.json()['data']])

In [23]:
# Note the "model": "query" field in the payload
query_payload = {
  "input": "Who is a physicist associated with atomic bomb?",
    "model" : "query",
    "encoding_format": "float"
}
query_response = session.post(invoke_url, headers=headers, json=query_payload)
while query_response.status_code == 202:
    request_id = query_response.headers.get("NVCF-REQID")
    fetch_url = fetch_url_format + request_id
    query_response = session.get(fetch_url, headers=headers)

query_response.raise_for_status()
query_embedding = np.asarray(query_response.json()['data'][0]['embedding'])

In [24]:
# A simple dot product
np.dot(passage_embeddings, query_embedding)

array([0.32027256, 0.5354134 , 0.6735917 ])